# EDA (Аналих датасетов распределний признаков в датасетах)

## Загрузим и импортируем нужные модули

In [1]:
!pip install -q deepchecks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.4 MB/s eta 0:00:00


In [9]:
!pip install -q pandas

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [3]:
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import FeatureDrift

## Выгрузим наши датасеты

In [4]:
data_anomaly = pd.read_csv("dataset._anomaly.csv", sep=";", decimal = ',')
data_normal = pd.read_csv("dataset._normal.csv", sep=";", decimal = ',')
data_problems = pd.read_csv("dataset._problems.csv", sep=";", decimal = ',')

<ipython-input-4-d1562647d5c9>:2: DtypeWarning:

Columns (1,3,5,12,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.

<ipython-input-4-d1562647d5c9>:3: DtypeWarning:

Columns (1,3,5,12,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.



## Первичный осмотр

In [ ]:
data_anomaly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189704 entries, 0 to 189703
Data columns (total 55 columns):
 #   Column                                                       Non-Null Count   Dtype  
---  ------                                                       --------------   -----  
 0   Дата и время                                                 189704 non-null  object 
 1   Полож.пед.акселер.,%                                         189704 non-null  float64
 2   Нагрузка на двигатель, %                                     189704 non-null  object 
 3   Давл.масла двиг.,кПа                                         189704 non-null  int64  
 4   Темп.масла двиг.,°С                                          189704 non-null  int64  
 5   Обор.двиг.,об/мин                                            189704 non-null  float64
 6   Значение счетчика моточасов, час:мин                         189704 non-null  object 
 7   Сост.пед.сцепл.                                              1897

In [33]:
data_problems.columns

Index(['Полож.пед.акселер.,%', 'Нагрузка на двигатель, %',
       'Давл.масла двиг.,кПа', 'Темп.масла двиг.,°С', 'Обор.двиг.,об/мин',
       'Значение счетчика моточасов, час:мин', 'Сост.пед.сцепл.', 'iButton2',
       'КПП. Температура масла', 'КПП. Давление масла в системе смазки',
       'Скорость', 'ДВС. Давление смазки',
       'ДВС. Температура охлаждающей жидкости',
       'Давление в пневмостистеме (spn46), кПа', 'Уровень топлива % (spn96)',
       'Электросистема. Напряжение', 'ДВС. Частота вращения коленчатого вала',
       'Нейтраль КПП (spn3843)', 'Стояночный тормоз (spn3842)',
       'Аварийная температура охлаждающей жидкости (spn3841)',
       'Засоренность воздушного фильтра (spn3840)',
       'Засоренность фильтра КПП (spn3847)',
       'Аварийное давление масла ДВС (spn3846)',
       'Засоренность фильтра ДВС (spn3845)',
       'Засоренность фильтра рулевого управления (spn3844)',
       'Засоренность фильтра навесного оборудования (spn3851)',
       'Недопустимый уро

## Приведем препроцессинг данных

In [5]:
features = [col for col in data_problems.columns if col not in ['Дата и время']]

In [6]:
data_normal.replace("-",np.nan,inplace=True)
data_normal.replace('        -',np.nan,inplace=True)
data_normal = data_normal[features].dropna(how='all')

data_problems.replace("-",np.nan,inplace=True)
data_problems.replace('        -',np.nan,inplace=True)
data_problems = data_problems[features].dropna(how='all')

data_anomaly.replace("-",np.nan,inplace=True)
data_anomaly.replace('        -',np.nan,inplace=True)
data_anomaly = data_anomaly[features].dropna(how='all')

data_normal['problem'] = np.array([0]*len(data_normal))
data_problems['problem'] = np.array([1]*len(data_problems))
data_anomaly['problem'] = np.array([0]*len(data_anomaly))

data_normal['anomaly'] = np.array([0]*len(data_normal))
data_problems['anomaly'] = np.array([0]*len(data_problems))
data_anomaly['anomaly'] = np.array([1]*len(data_anomaly))

In [7]:
metki = ['Нейтраль КПП (spn3843)', 'Стояночный тормоз (spn3842)',
       'Аварийная температура охлаждающей жидкости (spn3841)',
       'Засоренность воздушного фильтра (spn3840)',
       'Засоренность фильтра КПП (spn3847)',
       'Аварийное давление масла ДВС (spn3846)',
       'Засоренность фильтра ДВС (spn3845)',
       'Засоренность фильтра рулевого управления (spn3844)',
       'Засоренность фильтра навесного оборудования (spn3851)',
       'Недопустимый уровень масла в гидробаке (spn3850)',
       'Аварийная температура масла в гидросистеме (spn3849)',
       'Аварийное давление в I контуре тормозной системы (spn3848)',
       'Аварийное давление в II контуре тормозной системы (spn3855)',
       'Зарядка АКБ (spn3854)', 'Отопитель (spn3853)',
       'Выход блока управления двигателем (spn3852)',
       'Включение тормозков (spn3859)', 'Засоренность фильтра слива (spn3858)',
       'Аварийное давление масла КПП (spn3857)',
       'Аварийная температура масла ДВС(spn3856)',
       'Неисправность тормозной системы (spn3863)', 'Термостарт (spn3862)',
       'Разрешение запуска двигателя (spn3861)', 'Низкий уровень ОЖ (spn3860)',
       'Аварийная температура масла ГТР (spn3867)',
       'Необходимость сервисного обслуживания (spn3866)',
       'Подогрев топливного фильтра (spn3865)', 'Вода в топливе (spn3864)',
       'Холодный старт (spn3871)', 'Крутящий момент (spn513), Нм',
       'Положение рейки ТНВД (spn51), %', 'Расход топлива (spn183), л/ч',
       'ДВС. Температура наддувочного воздуха, °С',
       'Давление наддувочного воздуха двигателя (spn106), кПа',
       'Текущая передача (spn523)',
       'Температура масла гидравлики (spn5536), С', 'Педаль слива (spn598)']

In [12]:
def to_standart(x):
    try:
        return int(x)
    except:
        return 0

data_anomaly[metki] = data_anomaly[metki].applymap(lambda x: to_standart(x))
data_problems[metki] = data_problems[metki].applymap(lambda x: to_standart(x))
data_normal[metki] = data_normal[metki].applymap(lambda x: to_standart(x))

In [13]:
def to_time(x):
    try:
        return int(x[:3])*60+int(x[-2:])
    except:
        return np.nan
data_anomaly['Значение счетчика моточасов, час:мин'] = data_anomaly['Значение счетчика моточасов, час:мин'].map(lambda x: to_time(x))
data_problems['Значение счетчика моточасов, час:мин'] = data_problems['Значение счетчика моточасов, час:мин'].map(lambda x: to_time(x))
data_normal['Значение счетчика моточасов, час:мин'] = data_normal['Значение счетчика моточасов, час:мин'].map(lambda x: to_time(x))

In [14]:
num_feat = ['Полож.пед.акселер.,%', 'Нагрузка на двигатель, %',
       'Давл.масла двиг.,кПа', 'Темп.масла двиг.,°С', 'Обор.двиг.,об/мин',
       'Значение счетчика моточасов, час:мин',
       'КПП. Температура масла', 'КПП. Давление масла в системе смазки',
       'Скорость', 'ДВС. Давление смазки',
       'ДВС. Температура охлаждающей жидкости',
       'Давление в пневмостистеме (spn46), кПа', 'Уровень топлива % (spn96)',
       'Электросистема. Напряжение', 'ДВС. Частота вращения коленчатого вала']
cat_feat = ['iButton2','Сост.пед.сцепл.']
features = num_feat+cat_feat
len(features)

17

In [15]:
data_anomaly[cat_feat] = data_anomaly[cat_feat].fillna('no_data')
data_problems[cat_feat] = data_problems[cat_feat].fillna('no_data')
data_normal[cat_feat] = data_normal[cat_feat].fillna('no_data')
data_anomaly[cat_feat] = data_anomaly[cat_feat].astype("category")
data_problems[cat_feat] = data_problems[cat_feat].astype("category")
data_normal[cat_feat] = data_normal[cat_feat].astype("category")

In [23]:
for_rep = []
for feat in num_feat:
    try:
        data_normal[feat] = data_normal[feat].astype(float)
    except:
        for_rep.append(feat)

In [27]:
def to_int(x):
    return str(x).replace(',','.')
data_problems[for_rep] = data_problems[for_rep].applymap(lambda x: to_int(x))
data_normal[for_rep] = data_normal[for_rep].applymap(lambda x: to_int(x))

In [28]:
data_anomaly[num_feat] = data_anomaly[num_feat].astype("float64")
data_problems[num_feat] = data_problems[num_feat].astype("float64")
data_normal[num_feat] = data_normal[num_feat].astype("float64")

## Посмотрим на разницы распределений основных признаков в трех выборках:

### Разница между anomaly и normal данными

- Синий график: аномальные данные
- Бирюзовый график: нормальные данные

In [29]:
anomaly_dd = Dataset(
    data_anomaly[features],
    cat_features=cat_feat,
    features=features
)
normal_dd = Dataset(
    data_normal[features],
    cat_features=cat_feat,
    features=features
)

check = FeatureDrift(categorical_drift_method = 'PSI', n_top_columns = 15)
check_result = check.run(train_dataset=anomaly_dd, test_dataset=normal_dd)

check_result.show()

<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Feature Drift

### Разница между problems и normal данными

- Синий график: проблемные данные
- Бирюзовый график: нормальные данные

In [30]:
problems_dd = Dataset(
    data_problems[features],
    cat_features=cat_feat,
    features=features
)
normal_dd = Dataset(
    data_normal[features],
    cat_features=cat_feat,
    features=features
)

check = FeatureDrift(categorical_drift_method = 'PSI', n_top_columns = 15)
check_result = check.run(train_dataset=problems_dd, test_dataset=normal_dd)

check_result.show()

<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Feature Drift

### Разница между anomaly и problems данными

- Синий график: проблемные данные
- Бирюзовый график: аномальные данные

In [31]:
check = FeatureDrift(categorical_drift_method = 'PSI', n_top_columns = 15)
check_result = check.run(train_dataset=problems_dd, test_dataset=anomaly_dd)

check_result.show()

<!--
 ~ ----------------------------------------------------------------------------
 ~ Copyright (C) 2021-2023 Deepchecks (https://www.deepchecks.com)
 ~
 ~ This file is part of Deepchecks.
 ~ Deepchecks is distributed under the terms of the GNU Affero General
 ~ Public License (version 3 or later).
 ~ You should have received a copy of the GNU Affero General Public License
 ~ along with Deepchecks. If not, see .
 ~ ----------------------------------------------------------------------------
 ~
-->
<!DOCTYPE html>
 
 
 
 Feature Drift

Мы видим серьезные статистические различия между выборками, что говорит об их явных различиях и дает повод задуматься о причинах аномалий и поломок